## 1.2 - Выгрузка данных по просрочке

### 1.2.1 - Библиотеки

In [1]:
import pandas as pd
import numpy as np

# from utils import master_scale
from _utils.text_preporation import master_scale

from _utils.date_preprocessing import *

from tqdm.notebook import tqdm

import cx_Oracle
from sqlalchemy import create_engine

import yaml

import warnings
warnings.filterwarnings("ignore")

from SQLQueryHandler import SQLQueryHandler
from DatabaseManager import DatabaseManager

### 1.2.2 - Пути

In [2]:
with open('../CONFIGS.yaml', 'r') as file:
    config = yaml.safe_load(file)

with open('../DATABASE_CONFIG.yaml', 'r') as file:
    DB_CONFIG = yaml.safe_load(file)

PATHS = config['data_paths']
PARAMS = config['calibration_params']

filepath = {
      'clean_data' : PATHS['other']['clean_data']
    , 'valid_dates': PATHS['ratings']['valid_dates']
    , 'master_scale': PATHS['ratings']['master_scale']
    , 'compact_rating_data': PATHS['ratings']['preprocessed_compact']
    , 'rating_weights': PATHS['ratings']['weights']
    , 'dpd': PATHS['risk']['dpd_data']
}

### 1.2.3 - Данные

In [3]:
weight_col = 'W_norm'
target_col = 'target'

dpd_columns = [
    'rating_id',
    'rating_assignment_date',
    target_col,
    weight_col,
    'dpd'
]

In [4]:
def show_example(data, example_client=706021):
    return data.query('client_id == @example_client')

In [5]:
data = pd.read_parquet(filepath['clean_data'])
valid_dates = pd.read_parquet(filepath['valid_dates'])
rating_probs_range_df = pd.read_excel(filepath['master_scale'])
rating_probs_range_df.rename(columns = {
      'Рейтинг' : 'rating'
    , 'Нижний предел' : 'lower_bound'
    , 'Вероятность Дефолта' : 'pd'
    , 'Верхний предел' : 'upper_bound'
}, inplace=True)

df_clean = pd.read_parquet(filepath['clean_data'])
df_new_rating = pd.read_csv(filepath['compact_rating_data'], sep='|').drop(['Unnamed: 0'], axis=1)

dt_rating_weights = pd.read_parquet(filepath['rating_weights'])

kz_holidays = holidays.KZ()

In [6]:
db_manager = DatabaseManager( login=DB_CONFIG['login']
                            , password=DB_CONFIG['password']
                            , scheme=DB_CONFIG['scheme']
                            , host=DB_CONFIG['host']
                            , port=DB_CONFIG['port'])

### 1.2.4 - Формирование выборки

In [7]:
df_final = pd.merge(df_clean,
                    dt_rating_weights,
                    how = 'inner',
                    on = ['rating_id', 'client_id', 'rating_assignment_date', 'target'])

In [8]:
df_final['ssud_day'] = df_final['rating_assignment_date'].apply(lambda x: x.strftime('%Y-%m-%d'))
df_final['ssud_day'] = df_final['ssud_day'].apply(lambda x: get_closest_work_day(x))

df_final['str_date_u'] = df_final['ssud_day'].astype(str)

df_final['bin'] = df_final['bin'].astype(str).str.zfill(12)

### 1.2.5 - Выгрузка данных о просрочке

In [9]:
u_dpd_list = []

for i, j in tqdm(zip(df_final.bin, df_final.str_date_u)):
    query = """ 
        select 
        iinbin,
        report_date,
        max(day_pr_od) as dpd
        from  ods.mpr_loan_mart_ssud
        where 1=1
        and report_date = to_date('{}', 'YYYY-MM-DD')
        and iinbin = '{}'
        group by iinbin, report_date
        """.format(j, i)
    tmp = pd.read_sql(query, db_manager.engine, coerce_float=False)
    u_dpd_list.append(tmp)

u_dpd_df = pd.concat(u_dpd_list, ignore_index=True)

0it [00:00, ?it/s]

### 1.2.6 - Формирование датасета и проставление флагов

In [10]:
df_dpd = pd.merge(df_final,
                  u_dpd_df,
                  how='left',
                  left_on=['bin', 'rating_assignment_date'],
                  right_on=['iinbin', 'report_date'])

df_dpd['dpd'] = df_dpd['dpd'].fillna(0)


df_dpd = df_dpd[dpd_columns]
df_dpd = df_dpd.drop_duplicates()

df_dpd['0_dpd'] = np.where((df_dpd['dpd'] == 0), 1, 0)
df_dpd['1_5_dpd'] = np.where((df_dpd['dpd'] >= 1) & (df_dpd['dpd'] <= 5), 1, 0)
df_dpd['6_10_dpd'] = np.where((df_dpd['dpd'] >= 6) & (df_dpd['dpd'] <= 10), 1, 0)
df_dpd['11_15_dpd'] = np.where((df_dpd['dpd'] >= 11) & (df_dpd['dpd'] <= 15), 1, 0)
df_dpd['1_15_dpd'] = np.where((df_dpd['dpd'] >= 1) & (df_dpd['dpd'] <= 15), 1, 0)
df_dpd['5_15_dpd'] = np.where((df_dpd['dpd'] >= 5) & (df_dpd['dpd'] <= 15), 1, 0)
df_dpd['5_14_dpd'] = np.where((df_dpd['dpd'] >= 5) & (df_dpd['dpd'] <= 14), 1, 0)
df_dpd['14_29_dpd'] = np.where((df_dpd['dpd'] >= 15) & (df_dpd['dpd'] <= 29), 1, 0)
df_dpd['16_29_dpd'] = np.where((df_dpd['dpd'] >= 16) & (df_dpd['dpd'] <= 29), 1, 0)
df_dpd['30_59_dpd'] = np.where((df_dpd['dpd'] >= 30) & (df_dpd['dpd'] <= 59), 1, 0)

DR по просрочкам

In [11]:
df_dpd[[col for col in df_dpd.columns if '_dpd' in col]].mean().round(4)*100

0_dpd        96.31
1_5_dpd       1.12
6_10_dpd      1.42
11_15_dpd     0.16
1_15_dpd      2.71
5_15_dpd      1.65
5_14_dpd      1.65
14_29_dpd     0.44
16_29_dpd     0.44
30_59_dpd     0.54
dtype: float64

In [13]:
w = pd.DataFrame()

columns = df_dpd.columns[-10:]


yes = []
volume = []
target_yes = []
weight_sum = []
weight_target_yes = []
for i in columns:
    yes.append(df_dpd[df_dpd[i] == 1].shape[0])
    volume.append(df_dpd.shape[0])
    target_yes.append(df_dpd[df_dpd[i] == 1][target_col].sum())
    weight_sum.append(df_dpd[df_dpd[i] == 1][weight_col].sum())
    weight_target_yes.append(df_dpd[(df_dpd[i] == 1) & (df_dpd[target_col] == 1)][weight_col].sum())
    
    
w['signal'] = columns
w['volume'] = volume
w['abs_yes'] = yes
w['target_yes'] = target_yes
w['weight_sum'] = weight_sum
w['weight_target_yes'] = weight_target_yes


w['stat_raiting'] = w['target_yes'] / w['abs_yes']
w['stat_raiting'] = w['stat_raiting'].fillna(0)
w['stat_raiting'] = w['stat_raiting'].apply(lambda x: master_scale(x, rating_probs_range_df))
w['weighted_stat_raiting'] = w['weight_target_yes'] / w['weight_sum']
w['weighted_stat_raiting'] = w['weighted_stat_raiting'].fillna(0)
w['weighted_stat_raiting'] = w['weighted_stat_raiting'].apply(lambda x: master_scale(x, rating_probs_range_df))
w.fillna(0, inplace = True)

### 1.2.7 - Сохранение данных

In [ ]:
# df_dpd[['rating_id', 'dpd']].to_parquet(filepath['dpd'])